# Merge Beer_Wine_Spirits ML RF with Syndiago
  - Module name: Merge Beer_Wine_Spirits ML RF with Syndiago
  - Process 
  
    1). Process 'Beer_Wine_Spirits_ML and RF.xlsx' --> df_ml_rf
    - No missing value in GTIN  
    - There are 78912 unique GTIN
    - There are 3326 duplicate GTIN
    - read 'Beer_Wine_Spirits_ML and RF.xlsx' -> df_ml_rf_org  -> use index to create a column 'row_no'
    -  df_ml_rf_org - keep certain col ->  df_ml_rf
    
    2). process 'Syndigo 113k  Levels.xlsx' --> df_syndiago
      - No missing value in GTIN  
      - There are 67839 unique GTIN
      - There are 12692 duplicate GTIN
      - 'Syndigo 113k  Levels.xlsx' -> df_syndiago_org ; shape = (113898, 14)
      - two col names as 'GTIN' from 'Syndigo 113k  Levels.xlsx' 
      - df_syndiago_org will save then as 'GTIN' and  'GTIN.1'
        - All the gtin in 'GTIN' and  'GTIN.1' are identical 
      - df_syndiago_org -> df_syndiago_a (keep_col = ['GTIN', 'Level 1', 'Level 2','Level 3' ] ) 
      - df_syndiago_a -> df_syndiago (deduplicate GTIN;  shape = (113898, 14) )
        
     3). Merge df_ml_rf with  df_syndiago --> df_merge 
       - df_ml_rf +  df_syndiago - left out join with key = GTIN -> df_merge
       - df_merge -- reorder col ->  df_merge
       
     4). Validate df_merge  

In [1]:
import numpy as np
import pandas as pd 

pd.options.display.float_format = '{:,.4f}'.format
pd.set_option('display.max_columns', None) 
pd.set_option("display.max_rows", None)
path = 'C:\\users\\iny2819\\kroger\\Data\\'

## Process 'Beer_Wine_Spirits_ML and RF.xlsx' --> df_ml_rf

In [2]:

path_file = path + 'Beer_Wine_Spirits_ML and RF.xlsx'
df_ml_rf_org = pd.read_excel(path_file)
df_ml_rf_org = df_ml_rf_org.rename_axis('row_num').reset_index()
keep_col = ['row_num', 'GTIN','PMY_DPT_DSC', 'REC_DPT_DSC', 'DPT_DSC', 'COM_DSC', 'SUBCOM_DSC','VND_ECOM_DSC', 'ML L1 Predictions',
           'ML L2 Predictions', 'RF Level 1', 'RF Level 2', 'RF Level 3']
df_ml_rf = df_ml_rf_org[keep_col]


In [15]:
df_ml_rf.shape # (82580, 13)

(82580, 13)

In [16]:
gtin = df_ml_rf['GTIN'].value_counts(dropna = False)
len(gtin)  #  78912 unique gtin

78912

In [17]:
&gtin = gtin[gtin != 1]
len(gtin) #  duplicate gtin = 3326

3326

## process 'Syndigo 113k Levels.xlsx' --> df_syndiago

In [32]:
path_file = path + 'Syndigo_GTIN_Mapping_060723.xlsx'
df_syndiago_org = pd.read_excel(path_file)
df_syndiago_org.shape  #(113898, 14)


(113898, 13)

In [33]:
df_syndiago_org.columns

Index(['GTIN', 'Syndigo Taxonomy', 'Level Mapped', 'Level 1', 'Level 2',
       'Level 3', 'Level 4', 'Level 5', 'Level 6', 'Level 7', 'Level 8',
       'Level 9', 'Level 10'],
      dtype='object')

In [6]:
# check if values in  'GTIN' and 'GTIN.1' are all identical
#df_syndiago_org[df_syndiago_org['GTIN'] == df_syndiago_org['GTIN.1'] ].shape # (113898, 14)

(113898, 14)

In [34]:
# keep certain cols
keep_col = ['GTIN', 'Level 1', 'Level 2','Level 3' ]
df_syndiago_a = df_syndiago_org[keep_col] # (113898, 4)
df_syndiago_a.shape

(113898, 4)

In [35]:
"""
 1. find duplicate GTIN in df_syndiago_a
 2. there are 13 rows with gtin = 851718000000 with 13 duplicate  h
"""
gtin = df_syndiago_a['GTIN'].value_counts(dropna = False)
len(gtin)  # 95032 unique gtin

95032

In [36]:
gtin = gtin[gtin != 1]
len(gtin) #  duplicate gtin = 13444

13444

In [3]:
# display duplicate gtin
""" (duplicate)
   - 11110046208 (duplicate)
"""
gtin[0:10]  

NameError: name 'gtin' is not defined

In [38]:
# drop duplicates GTIN from df_syndiago_a
df_syndiago = df_syndiago_a.drop_duplicates(subset='GTIN', keep="first")
df_syndiago.shape # (95032, 4)

(95032, 4)

## Merge files

In [48]:
# Merge files
df_merge = df_ml_rf.merge(df_syndiago, on = 'GTIN', how = 'left') # LEFT OUTER JOIN
df_merge.shape # (82580, 16)

(82580, 16)

In [40]:
reo_col = ['row_num','GTIN', 'Level 1', 'Level 2','Level 3',   'RF Level 1', 'RF Level 2', 'RF Level 3','ML L1 Predictions',
           'ML L2 Predictions','PMY_DPT_DSC', 'REC_DPT_DSC', 'DPT_DSC', 'COM_DSC', 'SUBCOM_DSC','VND_ECOM_DSC']
df_merge = df_merge[reo_col]

In [41]:
# find duplicate GTIN after merge
gtin = df_merge['GTIN'].value_counts(dropna = False)
len(gtin)  # 78912

78912

In [42]:
gtin = gtin[gtin != 1]
len(gtin) #  duplicate gtin = 3326

3326

In [21]:
gtin[0:10]

72036820891       5
72036838384       5
88508848065       5
77890763674       5
89809103631       5
10083085420007    4
10083085904088    4
18200962721       4
850013004119      4
18200039973       4
Name: GTIN, dtype: int64

In [ ]:
# drop _duplicates GTIN from df_syndiago
df_syndiago_1 = df_syndiago.drop_duplicates(subset='GTIN', keep="first")

In [ ]:
 df_syndiago_1.shape # (113898, 4) # (67879, 4);  why not 95k

In [ ]:
merge_df = df_org.merge(df_syndiago, on = 'GTIN', how = 'left') # LEFT OUTER JOIN
#merge_df.shape

In [28]:
 merge_df.shape

(82580, 16)

In [ ]:
merge_df.shape

In [ ]:
# find duplicate GTIN after merge
gtin = merge_df['GTIN'].value_counts(dropna = False)
len(gtin)  # 78912

In [ ]:
gtin = gtin[gtin != 1]
len(gtin) #  duplicate gtin = 3340


In [ ]:
gtin [0:10]

In [ ]:
df_syndiago[df_syndiago['GTIN'] == 851718000000]

In [31]:
df_merge['Level 1'].value_counts(dropna = False)

NaN                      82579
Beer / Wine / Spirits        1
Name: Level 1, dtype: int64

In [29]:
df_merge[df_merge['GTIN'] == 72036820891]

,row_num,GTIN,Level 1,Level 2,Level 3,RF Level 1,RF Level 2,RF Level 3,ML L1 Predictions,ML L2 Predictions,PMY_DPT_DSC,REC_DPT_DSC,DPT_DSC,COM_DSC,SUBCOM_DSC,VND_ECOM_DSC
70285,70285,72036820891,NaN,NaN,NaN,Beer / Wine / Spirits,Wine,Still,Beer / Wine / Spirits,Wine,GROCERY,LIQUOR,WINE,TABLE WINE,750ML RED VALUE WINES $0-$7.99,) CAKEBREAD CHARDONNAY
70286,70286,72036820891,NaN,NaN,NaN,Beer / Wine / Spirits,Wine,Still,Beer / Wine / Spirits,Wine,GROCERY,LIQUOR,WINE,TABLE WINE,750ML RED VALUE WINES $0-$7.99,) CAKEBREAD CHARDONNAY
70287,70287,72036820891,NaN,NaN,NaN,Beer / Wine / Spirits,Wine,Still,Beer / Wine / Spirits,Wine,GROCERY,LIQUOR,WINE,TABLE WINE,750ML RED VALUE WINES $0-$7.99,) CAKEBREAD CHARDONNAY
70288,70288,72036820891,NaN,NaN,NaN,Beer / Wine / Spirits,Wine,Still,Beer / Wine / Spirits,Wine,GROCERY,LIQUOR,WINE,TABLE WINE,750ML RED VALUE WINES $0-$7.99,) CAKEBREAD CHARDONNAY
70289,70289,72036820891,NaN,NaN,NaN,Beer / Wine / Spirits,Beer,NaN,Beer / Wine / Spirits,Wine,GROCERY,LIQUOR,WINE,TABLE WINE,750ML RED VALUE WINES $0-$7.99,) CAKEBREAD CHARDONNAY


In [23]:
df_merge[df_merge['GTIN'] == 851718000000]  # no duplicate

,row_num,GTIN,PMY_DPT_DSC,REC_DPT_DSC,DPT_DSC,COM_DSC,SUBCOM_DSC,VND_ECOM_DSC,ML L1 Predictions,ML L2 Predictions,RF Level 1,RF Level 2,RF Level 3,Level 1,Level 2,Level 3
32644,32644,851718000000,GROCERY,LIQUOR,WINE,TABLE WINE,750ML RED LUXURY WINES ABOVE $15,Calista Coast Range Pinot Noir 25.4 Oz,Beer / Wine / Spirits,Wine,Beer / Wine / Spirits,Wine,Still,Beer / Wine / Spirits,Spirits,Whisky / Whiskey


In [65]:
df_syndiago[df_syndiago['GTIN'] == 82928206832]

,GTIN,Level 1,Level 2,Level 3


In [67]:
df_ml_rf[df_ml_rf['GTIN'] == 82928206832]

,row_num,GTIN,PMY_DPT_DSC,REC_DPT_DSC,DPT_DSC,COM_DSC,SUBCOM_DSC,VND_ECOM_DSC,ML L1 Predictions,ML L2 Predictions,RF Level 1,RF Level 2,RF Level 3
0,0,82928206832,GROCERY,LIQUOR,SPIRITS,TEQUILA,TRADITIONAL TEQUILA,El Toro Silver Tequila,Beer / Wine / Spirits,Spirits,Beer / Wine / Spirits,Spirits,Tequila / Mezcal


In [ ]:
df_merge[df_merge['GTIN'] == 851718000000]  # no duplicate

In [ ]:
#NW merge_df[merge_df['Level 1'] == pd.NaN].head(2)
merge_df[merge_df['Level 1'] == pd.nan].head(2)

In [ ]:

# merge_df.shape # (82609, 16); some GTIN might duplicate
df_org.shape  # (82580, 13)

In [ ]:
df_syndiago [df_syndiago['Level 1'] == 'Beer / Wine / Spirits'].shape  # (11899, 4)
df_syndiago.shape # (113898, 4)

In [ ]:
df_syndiago [df_syndiago['Level 1'] == 'Beer / Wine / Spirits'].value_counts()

In [ ]:
## WIP

In [ ]:
type(merge_df['Level 1'].value_counts(dropna = False)  ) # pandas.core.series.Series

In [47]:
df_merge['Level 1'].value_counts(dropna = False).sort_index() 

NaN    82580
Name: Level 1, dtype: int64

In [51]:
merge_df['Level 3'].value_counts(dropna = False)  # df_org has 84280 not match with Syndiago 

NaN                 82579
Whisky / Whiskey        1
Name: Level 3, dtype: int64

In [52]:
df_syndiago.dtypes

GTIN        int64
Level 1    object
Level 2    object
Level 3    object
dtype: object

In [54]:
df_ml_rf.dtypes

row_num               int64
GTIN                  int64
PMY_DPT_DSC          object
REC_DPT_DSC          object
DPT_DSC              object
COM_DSC              object
SUBCOM_DSC           object
VND_ECOM_DSC         object
ML L1 Predictions    object
ML L2 Predictions    object
RF Level 1           object
RF Level 2           object
RF Level 3           object
dtype: object

In [64]:
l_gtin_ml = df_ml_rf["GTIN"].to_list()

In [56]:
l_gtin_ml[0:10]

[82928206832,
 10082928206839,
 10012000027212,
 10012000151269,
 10051500286828,
 30051500286822,
 10078314211754,
 78314211757,
 10689352009984,
 10051500204747]

In [58]:
# drop duplicates GTIN from df_syndiago_a
df_mlrf_dedup = df_ml_rf.drop_duplicates(subset='GTIN', keep="first")
df_mlrf_dedup.shape # (78912, 4)

(78912, 13)

In [76]:
l_mlrf = sorted(df_mlrf_dedup['GTIN']) # lisr
type(l_mlrf) 

list

In [77]:
l_mlrf[0:10]

[222, 284, 765, 2004, 2011, 2035, 2042, 2066, 2073, 2080]

In [74]:
l_mlrf = df_mlrf_dedup['GTIN'].to_list()

type(l_mlrf)  # 78912 unique gtin

list

In [72]:
l_mlrf[0:10]

82928206832       1
40089854010067    1
50085000028211    1
401034000408      1
30085000023342    1
20085000024229    1
401034377968      1
40085000026562    1
30085000030968    1
30085000021324    1
Name: GTIN, dtype: int64

In [78]:
l_synd = sorted(df_syndiago['GTIN'])
l_synd[0:10]  # 95032 unique gtin


[56454666,
 80773443,
 80776307,
 80790990,
 80794233,
 111301201,
 338004183,
 338004404,
 370000051,
 513072921]

In [79]:
l_com = [x for x in l_mlrf if x in l_synd]
len(l_com) # 93

93

In [81]:
l_com[0:10] 

[11034410055,
 11034410079,
 11034420054,
 11034420078,
 11034430053,
 12354071995,
 21296602419,
 26861102974,
 26861103162,
 39383011587]

In [82]:
l_in_mlrf_not_in_synd  = [x for x in l_mlrf if x not in l_synd]
len(l_in_mlrf_not_in_synd) # 78819

93

In [84]:
len(l_in_mlrf_not_in_synd)

78819

In [83]:
l_in_mlrf_not_in_synd[0:10]

[222, 284, 765, 2004, 2011, 2035, 2042, 2066, 2073, 2080]